In [1]:
import ee
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import rasterio
ee.Initialize()

In [2]:
city_idx_start,city_idx_end=[0,400]
country_name="US"

city_400_file=f'/glade/u/home/yiwenz/TransferLearning/City_NameList/{country_name}_384.csv'
PATH_TO_HEIGHT_ROOT=f"/glade/work/yiwenz/building_height_{country_name}"
cities_400=pd.read_csv(city_400_file)
city_collection = f'projects/ee-yiwenz/assets/top_400_{country_name}_cities'
cities = ee.FeatureCollection(city_collection)

if not os.path.exists(PATH_TO_HEIGHT_ROOT):
    os.mkdir(PATH_TO_HEIGHT_ROOT)

"""
SRTM: The data represent “first-return” or Digital Surface Model data, which captures the elevation of the highest objects on the 
landscape, whether that is buildings, tree canopy, or the ground itself.
"""

In [7]:
############################# Building Height ############################
for city_idx in range(city_idx_start,city_idx_end):
    city_list=cities_400.loc[city_idx].to_list()
    city_name, _,=city_list
    city_name=int(city_name)

    PATH_TO_HEIGHT = os.path.join(PATH_TO_HEIGHT_ROOT,f'building_height_{city_name}.csv')
    if not os.path.exists(PATH_TO_HEIGHT):
        target_city= cities.filter(ee.Filter.eq('ORIG_FID',city_name))
        print(f'Downloading building height for city {str(city_name)}.')

        modis = ee.ImageCollection('MODIS/061/MYD11A1') \
                          .filterDate('2013-04-01')\
                        .select('LST_Day_1km')\
                        .first()
        gbh = ee.Image("users/yiwenz9/global_building_height").select(['b1']).rename(['height'])
        neighbor = gbh.focal_mean(1.5,'square','pixels',3)
        gbh_filled = gbh.unmask(neighbor).reproject(gbh.projection())
        gbh_filled_reproj=gbh_filled.reproject(modis.projection())

        gbh_vector=gbh_filled_reproj.sample(
                  region= target_city.geometry(),
                  dropNulls=False,
                  geometries=True,
                )
        gbh_list=gbh_vector.toList(gbh_vector.size())

        ghb_data = gbh_list.map(lambda feature: ee.Feature(feature).get('height')).getInfo()
        df_ghb = pd.DataFrame(ghb_data)
        nan_values = df_ghb.isnull().sum()[0]
        if nan_values !=0:
            print(nan_values)
        df_ghb.to_csv(PATH_TO_HEIGHT)
    else:
        print(f'Building height file already exists for {city_name}')

In [9]:
############################# Elevation ############################
from gee_prep import preprocess_modis, get_output_samples
import datetime

landsat_res = 30
radius = 16
modis_collection = 'MODIS/061/MYD11A1'
SRTM='USGS/SRTMGL1_003'
drive_folder=f'Elevation_Data_{country_name}'
img_export=ee.Image(SRTM)

for city_idx in range(city_idx_start,city_idx_end):
    city_list=cities_400.loc[city_idx].to_list()
    city_name, _,=city_list
    city_name=int(city_name)
    print(f'Exporting elevation for city {str(city_name)}.')
    
    target_city= cities.filter(ee.Filter.eq('ORIG_FID',city_name))
    modis=preprocess_modis(modis_collection, start_date='2013-04-01', end_date='2013-04-02', geometry=target_city)
    modis_download = modis.first()

    vector=modis_download.sample(
      region= target_city.geometry(),
      dropNulls=False,
      geometries=True,
    )
    img_export = img_export.reproject(modis.first().projection(),scale=30)
    collection_export = get_output_samples(img = img_export, region = vector, radius = radius, units = 'pixels', scale = landsat_res)
    description = f'elevation_{str(city_name)}'
    task = ee.batch.Export.table.toDrive(
        collection=collection_export,
        description=description,
        folder=drive_folder,
        fileNamePrefix=str(city_name),
        fileFormat='TFRecord')
    task.start()

Exporting elevation for city 42340.
Exporting elevation for city 24877.
Exporting elevation for city 51553.
Exporting elevation for city 35133.
Exporting elevation for city 46135.
Exporting elevation for city 46562.
Exporting elevation for city 33945.
Exporting elevation for city 38759.
Exporting elevation for city 25887.
Exporting elevation for city 34721.
Exporting elevation for city 28099.
Exporting elevation for city 51787.
Exporting elevation for city 51733.
Exporting elevation for city 41200.
Exporting elevation for city 24123.
Exporting elevation for city 51695.
Exporting elevation for city 28490.
Exporting elevation for city 16513.
Exporting elevation for city 16245.
Exporting elevation for city 34705.
Exporting elevation for city 23886.
Exporting elevation for city 28777.
Exporting elevation for city 43350.
Exporting elevation for city 40656.
Exporting elevation for city 38784.
Exporting elevation for city 41399.
Exporting elevation for city 27125.
Exporting elevation for city

In [185]:
def compute_impervious_fraction(img: ee.Image, feature: ee.Feature, radius: int, units: str, scale: float) -> ee.Feature:
    """
    Compute Impervious Surface Fraction (ISF) for a given feature.
    
    Args:
        img (ee.Image): ESA WorldCover image.
        feature (ee.Feature): The point feature (MODIS pixel).
        radius (int): Number of pixels to extend in each direction.
        units (str): Units for the kernel (e.g., 'meters').
        scale (float): Pixel resolution.

    Returns:
        ee.Feature: Feature with computed ISF value.
    """
    # Define a square kernel for neighborhood analysis
    kern = ee.Kernel.square(radius=radius, units=units)

    # Convert impervious surface (Class 50) to binary (1 = built-up, 0 = non-built-up)
    binary_impervious = img.eq(50).rename('impervious')

    # Compute the impervious fraction using a neighborhood mean
    isf = binary_impervious.reduceNeighborhood(
        reducer=ee.Reducer.mean(),  # Mean = Fraction of pixels classified as built-up
        kernel=kern,
        optimization='boxcar'  # Optimized processing
    ).rename('impervious')

    # Sample the computed ISF value at the feature's location
    sample = isf.sample(region=feature.geometry(), scale=scale).first().get('impervious')

    # Return feature with Impervious Surface Fraction (ISF) as a new property
    return feature.set('ISF', sample)


In [187]:
############################# Impervious Surface Fraction ############################
from gee_prep import preprocess_modis, get_output_samples
import datetime
import ee

modis_collection = 'MODIS/061/MYD11A1'
ESA_WORLDCOVER = 'ESA/WorldCover/v200'  # ESA WorldCover dataset
drive_folder = f'ISF_{country_name}'

# Load ESA WorldCover dataset
land_cover = ee.ImageCollection(ESA_WORLDCOVER).first().select('Map').reproject(ee.ImageCollection(modis_collection).first().projection(),scale=10)

for city_idx in range(0, 383):
    city_list = cities_400.loc[city_idx][['City_Name','NAME10']]
    city_name,NAME10, = city_list
    print(f'Exporting impervious surface fraction for city {str(city_name)}.')

    # Filter target city
    target_city = cities.filter(ee.Filter.eq('NAME10', NAME10))
    
    # Get MODIS pixels for this city
    modis = preprocess_modis(modis_collection, start_date='2013-04-01', end_date='2013-04-02', geometry=target_city)
    modis_download = modis.first()

 
    # Sample MODIS pixels
    vector = modis_download.sample(
        region=target_city.geometry(),
        dropNulls=False,
        geometries=True,
    )

    vector_with_fraction = vector.map(lambda feature: compute_impervious_fraction(
    img=land_cover, 
    feature=feature, 
    radius=500,  # MODIS pixel ~1km
    units='meters',
    scale=10  # ESA WorldCover resolution
)).select(['ISF'])
    description = f'ISF_{str(city_name)}'

    vector_data = vector_with_fraction.map(lambda f: ee.Feature(None, f.toDictionary()))
    
    task = ee.batch.Export.table.toDrive(
        collection=vector_data,
        description=description,
        folder=drive_folder,
        fileNamePrefix=description,
        fileFormat='CSV'
    )
    task.start()


Exporting impervious surface fraction for city NewYork_Newark_NY.
Exporting impervious surface fraction for city Atlanta_GA.
Exporting impervious surface fraction for city Chicago_IL.
Exporting impervious surface fraction for city Philadelphia_PA.
Exporting impervious surface fraction for city Boston_MA.
Exporting impervious surface fraction for city Dallas_FortWorth_Arlington_TX.
Exporting impervious surface fraction for city LosAngeles_LongBeach_Anaheim_CA.
Exporting impervious surface fraction for city Houston_TX.
Exporting impervious surface fraction for city Detroit_MI.
Exporting impervious surface fraction for city Washington_DC.
Exporting impervious surface fraction for city Miami_FL.
Exporting impervious surface fraction for city Phoenix_Mesa_AZ.
Exporting impervious surface fraction for city Minneapolis_StPaul_MN.
Exporting impervious surface fraction for city Seattle_WA.
Exporting impervious surface fraction for city Tampa_StPetersburg_FL.
Exporting impervious surface fractio

# Ignore below

In [ ]:
Map.addLayer(ee.Feature(modis_list.get(1000)),{},'vector_modis')
Map.addLayer(ee.Feature(gbh_list.get(1000)),{},'vector_gbh')

In [ ]:
task = ee.batch.Export.table.toAsset(
    collection=gbh_vector,
    selectors: ['id', 'height'],
    description='building_height',
    assetId='building_height_phoenix',
    fileFormat='csv')
task.start()

In [ ]:
vector=modis_reproj.sample(
          region= phoenix.geometry(),
          dropNulls=False,
          geometries=True,
        )
fname='AWS3D30_cropped'

In [ ]:
export_to_drive(img = srtm, region = vector, radius = 16,
                        units = 'pixels', scale = 30, folder = drive_folder, fname = fname)

In [ ]:
dataset=tf.data.TFRecordDataset(
    '021211.tfrecord.gz', compression_type='GZIP', buffer_size=None, num_parallel_reads=None
)

In [ ]:
bands=['Blue','Green','Red','NIR','SWIR1','SWIR2', 'ST', 'source_mask', 'qa_pixel_0', 'qa_radsat_0','qa_pixel_1', 'qa_radsat_1']
#bands=['Blue','Green','Red']
feature_names=list(bands)
feature_names

In [ ]:
# List of fixed-length features, all of which are float32.
predictor_columns = [
  tf.io.FixedLenFeature(shape=[33,33], dtype=tf.float32) for k in feature_names
]
target_columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(feature_names+[target], predictor_columns+target_columns))

pprint(features_dict)

In [ ]:
#Count the number of examples in a TFRecord
dataset.reduce(np.int64(0), lambda x, _: x + 1)

In [ ]:
def input_fn(serialized_example):

 # Make a parsing function
    example = tf.io.parse_single_example(serialized_example,features_dict)
    return example

parsed_dataset= dataset.map(input_fn)

In [ ]:
for elements in parsed_dataset.take(2114):
    print(elements)